In [302]:
from selenium import webdriver
import pandas as pd
from io import StringIO
from bs4 import BeautifulSoup
import re
import time
from datetime import date, timedelta, datetime
from webdriver_manager.chrome import ChromeDriverManager
from pprint import pprint

In [307]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/prupes/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


In [304]:
root = 'https://rotogrinders.com'

In [178]:
def get_base_page(date, Fanduel=False):
    if Fanduel:
        url = "https://rotogrinders.com/resultsdb/site/fanduel/date/{}/sport/nfl".format(date)
        return driver.get(url)
    else:
        url = "https://rotogrinders.com/resultsdb/site/draftkings/date/{}/sport/nfl".format(date)
        return driver.get(url)
def click_ownership_page():
    for elt in driver.find_elements_by_class_name("ant-tabs-tab"):
        if "Ownership" in elt.text:
            elt.click()
            return
    raise Exception('no button')
def click_summary_page():
    for elt in driver.find_elements_by_class_name("ant-tabs-tab"):
        if "Summary" in elt.text:
            elt.click()
            return
    raise Exception('no button')
def click_contests_page():
    for elt in driver.find_elements_by_class_name("ant-tabs-tab"):
        if "Contests" in elt.text:
            elt.click()
            return
    raise Exception('no button')
def click_right():
    driver.find_elements_by_css_selector('.ant-btn.ant-btn-icon-only')[1].click()
def get_winning_results():
    soup=BeautifulSoup(driver.page_source,'html.parser')
    tables = soup.findAll("table")
    table=pd.read_html(str(tables[0]))[0]
    for i in range(200):
        try:
            click_right()
        except:
            break
        time.sleep(.02)
        soup=BeautifulSoup(driver.page_source,'html.parser')
        tables = soup.findAll("table")
        a =pd.read_html(str(tables[0]))[0]
        table = table.append(a)
    return table
def get_actual_points():
    time.sleep(.3)
    click_ownership_page()
    time.sleep(1)
    soup=BeautifulSoup(driver.page_source,'html.parser')
    tables = soup.findAll("table")
    df = pd.read_html(str(tables[2]))[0]
    return df
def get_contests(date, max_contests=5):
    output = []
    time.sleep(.1)
    try:
        click_contests_page()
    except:
        return output
    time.sleep(.1)
    soup=BeautifulSoup(driver.page_source,'html.parser')
    tables = soup.findAll("table")
    #print(tables)
    index = 0 if len(tables) == 1 else 2
    rows = tables[index].find("tbody").find_all("tr")
    records = pd.read_html(str(tables[index]))[0].to_dict(orient='records')
    for i in range(min(len(rows),max_contests)):
        #print(rows[i].find_all('a', href=True))
        if float(records[i]['Top Prize'][1:].replace(",","")) > 4000 and float(records[i]['Buy In'][1:].replace(",","")) < 30 and float(records[i]['Max Entries']) >= 10: 
            print(root+rows[i].find_all('a', href=True)[0]['href'])
            driver.get(root+rows[i].find_all('a', href=True)[0]['href'])
            time.sleep(10)
            print(records[i])
            df = get_winning_results()
            records[i]['PAYOUTS']=df
            output.append(records[i])
    return output
def projections(date_string):
    output = {}
    url = 'https://rotogrinders.com/projected-stats/nfl?site=draftkings&date={}'.format(date_string)
    driver.get(url)
    time.sleep(3)
    #print('hello')
    soup=BeautifulSoup(driver.page_source,'html.parser')
    mydivs = soup.findAll("div", {"class": "rgt-col"})
    try:
        points = [float(a.text) for a in mydivs[-2].findAll("div")[1:]]
        name = [a.text for a in mydivs[0].findAll("div")[1:]]
        salary = [float(a.text[1:-1])*1000 for a in mydivs[1].findAll("div")[1:]]
        team = [a.text for a in mydivs[2].findAll("div")[1:]]
        position = [a.text for a in mydivs[3].findAll("div")[1:]]
        opp = [a.text for a in mydivs[4].findAll("div")[1:]]
        pownership = [float(a.text[:-1])/100 for a in mydivs[-10].findAll("div")[1:]]
    except:
        return False
    for i in range(len(name)):
        output[name[i]] = {'opp':opp[i], 'position': position[i], 'points':points[i],
                           'powner': pownership[i], 'salary':salary[i], 'team':team[i]}        #print(name[i], ceil[i], floor[i], points[i])
    return output
def scrape_data(date):
    try:
        get_base_page(date,Fanduel=False)
    except:
        return False
    time.sleep(8)
    try:
        actual_points = get_actual_points()
    except:
        return False
    contests = get_contests(date)
    projection = projections(date)
    return {"ACTUAL_POINTS": actual_points, "CONTESTS": contests, "PROJECTIONS": projection}
"""
def scrape_all_data(Sdate = date(2017,9,1),Edate = date.today()):
    huge_data = {}
    sdate = Sdate   
    edate = Edate
    #driver = webdriver.Chrome('/usr/local/bin/chromedriver')

    delta = edate - sdate       

    for i in range(delta.days + 1):
        day = sdate + timedelta(days=i)
        
        text = day.strftime("%Y-%m-%d")
        print(text)
        #huge_data[text] = scrape_data(text)
        result = scrape_data(text)
        if result != False:
            huge_data[text] = result
    return huge_data
"""
def scrape_all_data(dates):
    huge_data = {}
    for date in dates:
        result = scrape_data(date)
        if result != False:
            huge_data[date] = result
    return huge_data

In [234]:
latest = scrape_all_data(["2020-09-13"])

https://rotogrinders.com/resultsdb/site/draftkings/date/2020-09-13/sport/nfl/slate/5f5e57df5efd83599850c677/contest/5f5e57f818ea8c5997be95ef
{'Name': 'NFL $5M Fantasy Football Millionaire [$1M to 1st]', 'Link': 'Contest', 'Prize Pool': '$5,025,000', 'Buy In': '$5', 'Top Prize': '$1,000,000', 'Max Entries': 150, 'Entries': 1195006, 'Cash Line': nan, 'Winner': nan, 'Winning Score': nan}
https://rotogrinders.com/resultsdb/site/draftkings/date/2020-09-13/sport/nfl/slate/5f5e57df5efd83599850c677/contest/5f5e5a785efd835998531b4f
{'Name': 'NFL $1M Fantasy Football Million [$200K to 1st]', 'Link': 'Contest', 'Prize Pool': '$1,000,000', 'Buy In': '$5', 'Top Prize': '$200,000', 'Max Entries': 150, 'Entries': 237813, 'Cash Line': nan, 'Winner': nan, 'Winning Score': nan}
https://rotogrinders.com/resultsdb/site/draftkings/date/2020-09-13/sport/nfl/slate/5f5e57df5efd83599850c677/contest/5f5e5b045efd835998549b2f
{'Name': 'NFL $500K Play-Action [20 Entry Max]', 'Link': 'Contest', 'Prize Pool': '$500,

In [185]:
kaggle_df = pd.read_csv("../kaggle_resources/spreadspoke_scores.csv")

In [187]:
kaggle_df.tail()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
12673,01/12/2020,2019,Division,True,Green Bay Packers,28,23,Seattle Seahawks,GB,-4.5,45.5,Lambeau Field,False,NaN,NaN,NaN,NaN
12674,01/12/2020,2019,Division,True,Kansas City Chiefs,51,31,Houston Texans,KC,-10.0,50.5,Arrowhead Stadium,False,NaN,NaN,NaN,NaN
12675,01/19/2020,2019,Conference,True,Kansas City Chiefs,35,24,Tennessee Titans,KC,-7.0,51,Arrowhead Stadium,False,NaN,NaN,NaN,NaN
12676,01/19/2020,2019,Conference,True,San Francisco 49ers,37,20,Green Bay Packers,SF,-8.0,46.5,Levi's Stadium,False,NaN,NaN,NaN,NaN
12677,02/02/2020,2019,Superbowl,True,Kansas City Chiefs,31,20,San Francisco 49ers,KC,-1.5,53,Hard Rock Stadium,True,NaN,NaN,NaN,NaN


In [235]:
latest['2020-09-13']['PROJECTIONS']

{'Los Angeles Chargers': {'opp': '@ CIN', 'position': 'DST', 'points': 8.45, 'powner': 0.10300000000000001, 'salary': 2000.0, 'team': 'LAC'}, 'Tyrod Taylor': {'opp': '@ CIN', 'position': 'QB', 'points': 19.81, 'powner': 0.057999999999999996, 'salary': 5300.0, 'team': 'LAC'}, 'Detroit Lions': {'opp': 'CHI', 'position': 'DST', 'points': 7.07, 'powner': 0.047, 'salary': 2100.0, 'team': 'DET'}, 'Jared Goff': {'opp': 'DAL', 'position': 'QB', 'points': 19.5, 'powner': 0.0, 'salary': 5800.0, 'team': 'LAR'}, 'Patrick Mahomes': {'opp': 'HOU', 'position': 'QB', 'points': 25.66, 'powner': 0.0, 'salary': 7700.0, 'team': 'KCC'}, 'Irv Smith': {'opp': 'GBP', 'position': 'TE', 'points': 9.67, 'powner': 0.012, 'salary': 2900.0, 'team': 'MIN'}, 'Jimmy Garoppolo': {'opp': 'ARI', 'position': 'QB', 'points': 18.78, 'powner': 0.049, 'salary': 5700.0, 'team': 'SFO'}, 'Deshaun Watson': {'opp': '@ KCC', 'position': 'QB', 'points': 20.97, 'powner': 0.0, 'salary': 6400.0, 'team': 'HOU'}, 'New England Patriots': 

In [212]:
dates = kaggle_df.tail(1068).schedule_date
dates = [datetime.strptime(date, '%m/%d/%Y').strftime('%Y-%m-%d') for date in dates]
dates = sorted(list(set(dates)))

In [198]:
kaggle_df.tail(1068).schedule_date

11610    09/08/2016
11611    09/11/2016
11612    09/11/2016
11613    09/11/2016
11614    09/11/2016
            ...    
12673    01/12/2020
12674    01/12/2020
12675    01/19/2020
12676    01/19/2020
12677    02/02/2020
Name: schedule_date, Length: 1068, dtype: object

In [209]:
len(dates)

224

In [ ]:
RESULT = scrape_all_data(dates)

https://rotogrinders.com/resultsdb/site/draftkings/date/2016-09-08/sport/nfl/slate/5b4643a7edf229569ed8b0fb/contest/5b464778edf229569ee08909
{'Name': 'NFL $400K Gridiron [$400,000 Guaranteed] (Thu)', 'Link': 'Contest', 'Prize Pool': '$400,000', 'Buy In': '$27', 'Top Prize': '$50,000', 'Max Entries': 150, 'Entries': 15641, 'Cash Line': 151.28, 'Winner': 'fieldman77', 'Winning Score': 234.62}
https://rotogrinders.com/resultsdb/site/draftkings/date/2016-09-08/sport/nfl/slate/5b4643a7edf229569ed8b0fb/contest/5b464d14d37ec356d15fdd4d
{'Name': 'NFL $100K Slant [$100,000 Guaranteed] (Thu)', 'Link': 'Contest', 'Prize Pool': '$100,000', 'Buy In': '$9', 'Top Prize': '$5,000', 'Max Entries': 150, 'Entries': 10542, 'Cash Line': 145.92, 'Winner': 'fieldman77', 'Winning Score': 234.62}
https://rotogrinders.com/resultsdb/site/draftkings/date/2016-09-11/sport/nfl/slate/5b4643a4edf229569ed8b0f9/contest/5b46dcf9db9f875d23e84db5
{'Name': 'NFL $2.5M Sunday Special [$250,000 to 1st]', 'Link': 'Contest', 'P

https://rotogrinders.com/resultsdb/site/draftkings/date/2016-09-29/sport/nfl/slate/5b469e9a1dcee45a8cdc479c/contest/5b46a901bd89d35ad9200683
{'Name': 'NFL $350K Kickoff Special [$350,000 Guaranteed] (Thu)', 'Link': 'Contest', 'Prize Pool': '$350,000', 'Buy In': '$4', 'Top Prize': '$25,000', 'Max Entries': 150, 'Entries': 102941, 'Cash Line': 144.82, 'Winner': 'dtn0018', 'Winning Score': 239.12}
https://rotogrinders.com/resultsdb/site/draftkings/date/2016-09-29/sport/nfl/slate/5b469e9a1dcee45a8cdc479c/contest/5b46a74320600d5abc352929
{'Name': 'NFL $150K Gridiron [$150,000 Guaranteed] (Thu)', 'Link': 'Contest', 'Prize Pool': '$150,000', 'Buy In': '$27', 'Top Prize': '$10,000', 'Max Entries': 150, 'Entries': 6422, 'Cash Line': 148.22, 'Winner': 'Sqweatybalz', 'Winning Score': 227.7}
https://rotogrinders.com/resultsdb/site/draftkings/date/2016-10-02/sport/nfl/slate/5b4643d5edf229569ed95412/contest/5b4656445631f2571cd742ac
{'Name': 'NFL $500K Slant [$500,000 Guaranteed]', 'Link': 'Contest',

https://rotogrinders.com/resultsdb/site/draftkings/date/2016-10-20/sport/nfl/slate/5b4643e8edf229569ed98b84/contest/5b46d2f12a34965cdd414af2
{'Name': 'NFL $125K Gridiron [$125,000 Guaranteed] (Thu)', 'Link': 'Contest', 'Prize Pool': '$125,000', 'Buy In': '$27', 'Top Prize': '$8,000', 'Max Entries': 150, 'Entries': 5352, 'Cash Line': 157.52, 'Winner': 'draftdis | stenz13', 'Winning Score': 220.02}
https://rotogrinders.com/resultsdb/site/draftkings/date/2016-10-23/sport/nfl/slate/5b464daed37ec356d16222c9/contest/5b6be52b14f1998839eca57a
{'Name': 'NFL $500K Slant [$500,000 Guaranteed]', 'Link': 'Contest', 'Prize Pool': '$500,000', 'Buy In': '$9', 'Top Prize': '$25,000', 'Max Entries': 150, 'Entries': 64226, 'Cash Line': 161.22, 'Winner': 'Regilator1', 'Winning Score': 243.92}
https://rotogrinders.com/resultsdb/site/draftkings/date/2016-10-23/sport/nfl/slate/5b464daed37ec356d16222c9/contest/5b6be55c14f1998839eda1fa
{'Name': 'NFL $500K Slant [$500,000 Guaranteed]', 'Link': 'Contest', 'Prize

https://rotogrinders.com/resultsdb/site/draftkings/date/2016-11-17/sport/nfl/slate/5b4647a5edf229569ee0fb3f/contest/5b46e16fdb9f875d23fb3e58
{'Name': 'NFL $110K Gridiron [$110,000 Guaranteed] (Thu)', 'Link': 'Contest', 'Prize Pool': '$110,000', 'Buy In': '$27', 'Top Prize': '$7,000', 'Max Entries': 125, 'Entries': 4709, 'Cash Line': 135.5, 'Winner': 'papagates', 'Winning Score': 210.6}
https://rotogrinders.com/resultsdb/site/draftkings/date/2016-11-20/sport/nfl/slate/5b465ebb37861e575a3c3e64/contest/5b46a5d920600d5abc2fafc3
{'Name': 'NFL $4M Fantasy Football Millionaire [$1M to 1st]', 'Link': 'Contest', 'Prize Pool': '$4,000,000', 'Buy In': '$27', 'Top Prize': '$1,000,000', 'Max Entries': 150, 'Entries': 166641, 'Cash Line': 137.3, 'Winner': 'Moykiss', 'Winning Score': 223.1}
https://rotogrinders.com/resultsdb/site/draftkings/date/2016-11-20/sport/nfl/slate/5b465ebb37861e575a3c3e64/contest/5b46d4442a34965cdd462df6
{'Name': 'NFL $450K Slant [$450,000 Guaranteed]', 'Link': 'Contest', 'Pr

{'Name': 'NFL $350K Monday Night Showdown [$25K to 1st] (Mon-Thu)', 'Link': 'Contest', 'Prize Pool': '$350,000', 'Buy In': '$5', 'Top Prize': '$25,000', 'Max Entries': 150, 'Entries': 82352, 'Cash Line': 119.64, 'Winner': 'kruprock', 'Winning Score': 171.94}
https://rotogrinders.com/resultsdb/site/draftkings/date/2016-12-12/sport/nfl/slate/5b464414edf229569eda35d6/contest/5b71f155329a8645de17a0d8
{'Name': 'NFL $100K Gridiron [$100,000 Guaranteed] (Mon-Thu)', 'Link': 'Contest', 'Prize Pool': '$100,000', 'Buy In': '$27', 'Top Prize': '$5,000', 'Max Entries': 125, 'Entries': 4281, 'Cash Line': 116.64, 'Winner': 'dynodog', 'Winning Score': 162.76}
https://rotogrinders.com/resultsdb/site/draftkings/date/2016-12-15/sport/nfl/slate/5b464a79edf229569eec667b/contest/5b46b61b9a55fd5c1f8fa2ac
{'Name': 'NFL $150K Kickoff Special [$150,000 Guaranteed] (Thu)', 'Link': 'Contest', 'Prize Pool': '$150,000', 'Buy In': '$4', 'Top Prize': '$10,000', 'Max Entries': 150, 'Entries': 44117, 'Cash Line': 136.4

https://rotogrinders.com/resultsdb/site/draftkings/date/2017-01-08/sport/nfl/slate/5b464488edf229569edc78c4/contest/5b46602342460a576a36d47c
{'Name': 'NFL $500K Front Four [$500,000 Guaranteed] (Sunday Only)', 'Link': 'Contest', 'Prize Pool': '$500,000', 'Buy In': '$4', 'Top Prize': '$50,000', 'Max Entries': 150, 'Entries': 147058, 'Cash Line': 140.46, 'Winner': 'mocheez | bigtrig75 | 2incher', 'Winning Score': 206.98}
https://rotogrinders.com/resultsdb/site/draftkings/date/2017-01-08/sport/nfl/slate/5b464488edf229569edc78c4/contest/5b4677e3b26fdc589faa244d
{'Name': 'NFL $100K Front Four [$100,000 Guaranteed] (Sunday Only)', 'Link': 'Contest', 'Prize Pool': '$100,000', 'Buy In': '$4', 'Top Prize': '$5,000', 'Max Entries': 150, 'Entries': 29411, 'Cash Line': 139.78, 'Winner': 'Breeon', 'Winning Score': 206.98}
https://rotogrinders.com/resultsdb/site/draftkings/date/2017-01-14/sport/nfl/slate/5b4648cdedf229569ee5e6d4/contest/5b467e29a4052f58cd9dfd05
{'Name': 'NFL $2M Fantasy Football Qua

https://rotogrinders.com/resultsdb/site/draftkings/date/2017-09-28/sport/nfl/slate/5ae4dc4a78af173dfe001dd4/contest/5ae51b80589eaa73374fe584
{'Name': 'NFL $200K Play-Action [20 Entry Max] (Thu-Mon)', 'Link': 'Contest', 'Prize Pool': '$200,000', 'Buy In': '$3', 'Top Prize': '$10,000', 'Max Entries': 20, 'Entries': 79270, 'Cash Line': 124.66, 'Winner': 'arfw', 'Winning Score': 208.92}
https://rotogrinders.com/resultsdb/site/draftkings/date/2017-10-01/sport/nfl/slate/5ae4c8f7c074cf3dff2efb2a/contest/5ae63082940f5a20907701af
{'Name': 'NFL $4.25M Fantasy Football Millionaire [$1M to 1st]', 'Link': 'Contest', 'Prize Pool': '$4,250,000', 'Buy In': '$20', 'Top Prize': '$1,000,000', 'Max Entries': 150, 'Entries': 11250225003, 'Cash Line': 118.04, 'Winner': 'ChiChiWobbler', 'Winning Score': 219.4}
https://rotogrinders.com/resultsdb/site/draftkings/date/2017-10-01/sport/nfl/slate/5ae4c8f7c074cf3dff2efb2a/contest/5ae4d23c95356d3dfb4ca329
{'Name': 'NFL $500K Post Pattern', 'Link': 'Contest', 'Prize

{'Name': 'NFL $175K Play-Action [20 Entry Max] (Thu-Mon)', 'Link': 'Contest', 'Prize Pool': '$175,000', 'Buy In': '$3', 'Top Prize': '$10,000', 'Max Entries': 20, 'Entries': 69361, 'Cash Line': 124.08, 'Winner': 'TheMockingbird1', 'Winning Score': 243.98}
https://rotogrinders.com/resultsdb/site/draftkings/date/2017-10-29/sport/nfl/slate/5ae4c9085689223dfaf822fb/contest/5ae50775589eaa7337140947
{'Name': 'NFL $3.6M Fantasy Football Millionaire [$1M to 1st]', 'Link': 'Contest', 'Prize Pool': '$3,600,000', 'Buy In': '$20', 'Top Prize': '$1,000,000', 'Max Entries': 150, 'Entries': 200241, 'Cash Line': 131.02, 'Winner': 'cubsfan333', 'Winning Score': 247.88}
https://rotogrinders.com/resultsdb/site/draftkings/date/2017-10-29/sport/nfl/slate/5ae4c9085689223dfaf822fb/contest/5ae4de8a5689223dfa295c08
{'Name': 'NFL $350K Post Pattern', 'Link': 'Contest', 'Prize Pool': '$350,000', 'Buy In': '$8', 'Top Prize': '$30,000', 'Max Entries': 150, 'Entries': 51470, 'Cash Line': 134.42, 'Winner': 'cubsfan3

{'Name': 'NFL $225K Kickoff Special [$50K to 1st] (Thu-Mon)', 'Link': 'Contest', 'Prize Pool': '$225,000', 'Buy In': '$8', 'Top Prize': '$50,000', 'Max Entries': 150, 'Entries': 31851, 'Cash Line': 129.04, 'Winner': 'ChipotleAddict | ChipotleAddict', 'Winning Score': 229.14}
https://rotogrinders.com/resultsdb/site/draftkings/date/2017-12-03/sport/nfl/slate/5ae4c94fd69dcc3dfdd356fd/contest/5ae4cc2bd69dcc3dfdd95ccf
{'Name': 'NFL $3.15M Fantasy Football Millionaire [$1M to 1st]', 'Link': 'Contest', 'Prize Pool': '$3,150,000', 'Buy In': '$20', 'Top Prize': '$1,000,000', 'Max Entries': 150, 'Entries': 8637640572, 'Cash Line': 139.8, 'Winner': 'chess_is_ok', 'Winning Score': 241.14}
https://rotogrinders.com/resultsdb/site/draftkings/date/2017-12-03/sport/nfl/slate/5ae4c94fd69dcc3dfdd356fd/contest/5ae4c9dbf69c5a3e00141e2c
{'Name': 'NFL $1.25M Play-Action [$100K to 1st]', 'Link': 'Contest', 'Prize Pool': '$1,250,000', 'Buy In': '$3', 'Top Prize': '$100,000', 'Max Entries': 20, 'Entries': 49544

https://rotogrinders.com/resultsdb/site/draftkings/date/2017-12-31/sport/nfl/slate/5ae4c967dd71203dfc39fd10/contest/5ae5f158fd67640e26c780e9
{'Name': 'NFL $1.75M Mini 80th Special [$200K to 1st]', 'Link': 'Contest', 'Prize Pool': '$1,750,000', 'Buy In': '$8', 'Top Prize': '$200,000', 'Max Entries': 150, 'Entries': 257352, 'Cash Line': 137.14, 'Winner': 'Powermoves', 'Winning Score': 222.62}
https://rotogrinders.com/resultsdb/site/draftkings/date/2017-12-31/sport/nfl/slate/5ae4c967dd71203dfc39fd10/contest/5ae5ff933825540e24b2568b
{'Name': 'NFL $400K Play-Action', 'Link': 'Contest', 'Prize Pool': '$400,000', 'Buy In': '$3', 'Top Prize': '$25,000', 'Max Entries': 20, 'Entries': 158541, 'Cash Line': 134.08, 'Winner': 'isaacvanhook', 'Winning Score': 227.34}
https://rotogrinders.com/resultsdb/site/draftkings/date/2018-01-06/sport/nfl/slate/5ae4aa7f0f94ac3e011efc58/contest/5ae4bb590f94ac3e0154ddec
{'Name': 'NFL $2M Fantasy Football Quarter Millionaire [$250K to 1st]', 'Link': 'Contest', 'Pri

In [210]:
x = scrape_all_data(dates[-20:])

https://rotogrinders.com/resultsdb/site/draftkings/date/2017-10-09/sport/nfl/slate/5ae4ca5d0f94ac3e017cb75c/contest/5ae4cea0c074cf3dff398d48
{'Name': 'NFL $500K Monday Night Showdown [$50K to 1st] (Mon-Thu)', 'Link': 'Contest', 'Prize Pool': '$500,000', 'Buy In': '$5', 'Top Prize': '$50,000', 'Max Entries': 150, 'Entries': 118906, 'Cash Line': 36.88, 'Winner': 'Jeeber', 'Winning Score': 90.68}
https://rotogrinders.com/resultsdb/site/draftkings/date/2017-10-09/sport/nfl/slate/5ae4ca5d0f94ac3e017cb75c/contest/5ae4e496c074cf3dff72969e
{'Name': 'NFL $50K Flea Flicker (Mon-Thu)', 'Link': 'Contest', 'Prize Pool': '$50,000', 'Buy In': '$5', 'Top Prize': '$5,000', 'Max Entries': 150, 'Entries': 11890, 'Cash Line': 107.24, 'Winner': 'xrandy | hateonme', 'Winning Score': 150.46}
https://rotogrinders.com/resultsdb/site/draftkings/date/2016-11-03/sport/nfl/slate/5b464c27d37ec356d15c1f23/contest/5b46c36c34cc635c5f0ba787
{'Name': 'NFL $300K Kickoff Special [$300,000 Guaranteed] (Thu)', 'Link': 'Cont

https://rotogrinders.com/resultsdb/site/draftkings/date/2018-10-01/sport/nfl/slate/5bbbac3177d0ce599c1d6034/contest/5bd2050307a90936d13071db
{'Name': 'NFL Showdown $200K Play-Action [20 Entry Max] (KC vs DEN)', 'Link': 'Contest', 'Prize Pool': '$200,000', 'Buy In': '$3', 'Top Prize': '$10,000', 'Max Entries': 20, 'Entries': 79271, 'Cash Line': 97.6, 'Winner': 'jdkemp55 | bgt91178 | cwheels54 | mfraley | cjk23 | lildezzy11', 'Winning Score': 132.51}
https://rotogrinders.com/resultsdb/site/draftkings/date/2016-09-25/sport/nfl/slate/5b464ae9edf229569eedee57/contest/5b6a9a87cc7ecd6039cdfdcd
{'Name': 'NFL $450K Slant [$450,000 Guaranteed]', 'Link': 'Contest', 'Prize Pool': '$450,000', 'Buy In': '$9', 'Top Prize': '$25,000', 'Max Entries': 150, 'Entries': 57803, 'Cash Line': nan, 'Winner': nan, 'Winning Score': nan}
https://rotogrinders.com/resultsdb/site/draftkings/date/2016-09-25/sport/nfl/slate/5b464ae9edf229569eedee57/contest/5b46940808c0da5a3a1240af
{'Name': 'NFL $300K Flea Flicker [$30

https://rotogrinders.com/resultsdb/site/draftkings/date/2018-09-30/sport/nfl/slate/5bb10388ed767f5afae3935e/contest/5bb146dd4a2132753111b42e
{'Name': 'NFL $4M Roman Fantasy Football Millionaire [$1M to 1st]', 'Link': 'Contest', 'Prize Pool': '$4,000,000', 'Buy In': '$10', 'Top Prize': '$1,000,000', 'Max Entries': 150, 'Entries': 470588, 'Cash Line': 182.78, 'Winner': 'Nickelback4Lyfe', 'Winning Score': 273.56}
https://rotogrinders.com/resultsdb/site/draftkings/date/2018-09-30/sport/nfl/slate/5bb10388ed767f5afae3935e/contest/5bb1057ded767f5afaebe5c7
{'Name': 'NFL $750K Play-Action [20 Entry Max]', 'Link': 'Contest', 'Prize Pool': '$750,000', 'Buy In': '$3', 'Top Prize': '$75,000', 'Max Entries': 20, 'Entries': 297265, 'Cash Line': 178.9, 'Winner': 'ddowd84', 'Winning Score': 272.46}
https://rotogrinders.com/resultsdb/site/draftkings/date/2016-11-14/sport/nfl/slate/5b464ebc277cc056e65f719d/contest/5b46949508c0da5a3a1468f3
{'Name': 'NFL $300K Monday Night Showdown [$300,000 Guaranteed] (M

In [211]:
x.keys()

dict_keys(['2017-10-09', '2016-11-03', '2017-12-14', '2017-09-11', '2018-11-29', '2018-12-16', '2016-10-17', '2017-10-05', '2019-11-25', '2018-10-01', '2016-09-25', '2019-11-04', '2016-10-13', '2016-12-11', '2018-09-13', '2018-11-22', '2016-09-19', '2018-09-30', '2016-11-14', '2019-12-16'])

In [219]:
x['2017-12-14']['PROJECTIONS']

{}

In [221]:
for key in x:
    print(len(x[key]['PROJECTIONS']))

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [225]:
projections('2019-11-12')

{}

In [226]:
x.keys()

dict_keys(['2017-10-09', '2016-11-03', '2017-12-14', '2017-09-11', '2018-11-29', '2018-12-16', '2016-10-17', '2017-10-05', '2019-11-25', '2018-10-01', '2016-09-25', '2019-11-04', '2016-10-13', '2016-12-11', '2018-09-13', '2018-11-22', '2016-09-19', '2018-09-30', '2016-11-14', '2019-12-16'])

In [227]:
x['2019-12-16']['PROJECTIONS']

{}

In [236]:
driver.get("https://rotogrinders.com/lineuphq/nfl?site=draftkings")

In [274]:
soup=BeautifulSoup(driver.page_source,'html.parser')

In [243]:
table = driver.find_elements_by_class_name("fixedDataTableRowLayout_rowWrapper")

In [266]:
table = driver.find_elements_by_class_name("input-editable-container")

In [269]:
print(table[1].get_attribute('value'))

None


In [263]:
for elt in table:
    print(elt.text)

In [249]:
table1 = driver.find_elements_by_class_name("fixedDataTableCellGroupLayout_cellGroup")

In [258]:
table1[9].text

'Saquon Barkley\n$8.4K\n--'

In [240]:
tables = soup.findAll("table")

In [241]:
tables

[]

In [ ]:
<div class="fixedDataTableRowLayout_rowWrapper" style="width: 845px; height: 36px; z-index: 0; display: block; transform: translate3d(0px, 72px, 0px);"><div class="fixedDataTableRowLayout_main public_fixedDataTableRow_main public_fixedDataTableRow_even row-body row-0 public_fixedDataTable_bodyRow" role="row" aria-rowindex="3" style="width: 845px; height: 36px;"><div class="fixedDataTableRowLayout_body"><div class="fixedDataTableCellGroupLayout_cellGroupWrapper" style="height: 36px; width: 309px; left: 0px;"><div class="fixedDataTableCellGroupLayout_cellGroup" style="height: 36px; position: absolute; width: 309px; z-index: 2; transform: translate3d(0px, 0px, 0px);"><div class="fixedDataTableCellLayout_main public_fixedDataTableCell_main" role="gridcell" style="height: 36px; width: 140px; left: 0px;"><div class="fixedDataTableCellLayout_wrap1 public_fixedDataTableCell_wrap1 player-name-cell"><div class="fixedDataTableCellLayout_wrap2 public_fixedDataTableCell_wrap2"><div class="fixedDataTableCellLayout_wrap3 public_fixedDataTableCell_wrap3"><div class="public_fixedDataTableCell_cellContent"><span><a style="cursor: pointer;">Christian McCaffrey</a></span></div></div></div></div></div><div class="fixedDataTableCellLayout_main public_fixedDataTableCell_main" role="gridcell" style="height: 36px; width: 55px; left: 140px;"><div class="fixedDataTableCellLayout_wrap1 public_fixedDataTableCell_wrap1" style="white-space: nowrap;"><div class="fixedDataTableCellLayout_wrap2 public_fixedDataTableCell_wrap2"><div class="fixedDataTableCellLayout_wrap3 public_fixedDataTableCell_wrap3"><div class="public_fixedDataTableCell_cellContent">$10.0K</div></div></div></div></div><div class="fixedDataTableCellLayout_main public_fixedDataTableCell_main" role="gridcell" style="height: 36px; width: 55px; left: 195px;"><div class="fixedDataTableCellLayout_wrap1 public_fixedDataTableCell_wrap1"><div class="fixedDataTableCellLayout_wrap2 public_fixedDataTableCell_wrap2"><div class="fixedDataTableCellLayout_wrap3 public_fixedDataTableCell_wrap3"><div class="public_fixedDataTableCell_cellContent"><button type="button" class="modal-comp-link"><span class="notes-modal-icon icn-pencil"></span></button></div></div></div></div></div><div class="fixedDataTableCellLayout_main public_fixedDataTableCell_main" role="gridcell" style="height: 36px; width: 59px; left: 250px;"><div class="fixedDataTableCellLayout_wrap1 public_fixedDataTableCell_wrap1"><div class="fixedDataTableCellLayout_wrap2 public_fixedDataTableCell_wrap2"><div class="fixedDataTableCellLayout_wrap3 public_fixedDataTableCell_wrap3"><div class="public_fixedDataTableCell_cellContent"><span>--</span></div></div></div></div></div></div></div><div class="fixedDataTableCellGroupLayout_cellGroupWrapper" style="height: 36px; width: 349px; left: 309px;"><div class="fixedDataTableCellGroupLayout_cellGroup" style="height: 36px; position: absolute; width: 3121px; z-index: 0; transform: translate3d(0px, 0px, 0px);"><div class="fixedDataTableCellLayout_main public_fixedDataTableCell_main" role="gridcell" style="height: 36px; width: 74px; left: 0px;"><div class="fixedDataTableCellLayout_wrap1 public_fixedDataTableCell_wrap1 col-group-first"><div class="fixedDataTableCellLayout_wrap2 public_fixedDataTableCell_wrap2"><div class="fixedDataTableCellLayout_wrap3 public_fixedDataTableCell_wrap3"><div class="public_fixedDataTableCell_cellContent"><span>RB</span></div></div></div></div></div><div class="fixedDataTableCellLayout_main public_fixedDataTableCell_main" role="gridcell" style="height: 36px; width: 51px; left: 74px;"><div class="fixedDataTableCellLayout_wrap1 public_fixedDataTableCell_wrap1" style="white-space: nowrap;"><div class="fixedDataTableCellLayout_wrap2 public_fixedDataTableCell_wrap2"><div class="fixedDataTableCellLayout_wrap3 public_fixedDataTableCell_wrap3"><div class="public_fixedDataTableCell_cellContent">CAR</div></div></div></div></div><div class="fixedDataTableCellLayout_main public_fixedDataTableCell_main" role="gridcell" style="height: 36px; width: 65px; left: 125px;"><div class="fixedDataTableCellLayout_wrap1 public_fixedDataTableCell_wrap1 col-group-last" style="white-space: nowrap;"><div class="fixedDataTableCellLayout_wrap2 public_fixedDataTableCell_wrap2"><div class="fixedDataTableCellLayout_wrap3 public_fixedDataTableCell_wrap3"><div class="public_fixedDataTableCell_cellContent">@ TBB</div></div></div></div></div><div class="fixedDataTableCellLayout_main public_fixedDataTableCell_main" role="gridcell" style="height: 36px; width: 55px; left: 190px;"><div><span data-tip="false" data-for="rg-projection-fpts-0"><div class="fixedDataTableCellLayout_wrap1 public_fixedDataTableCell_wrap1 input-cell col-group-first"><div class="fixedDataTableCellLayout_wrap2 public_fixedDataTableCell_wrap2"><div class="fixedDataTableCellLayout_wrap3 public_fixedDataTableCell_wrap3"><div class="public_fixedDataTableCell_cellContent"><div class="input-editable-container"><input class="input-editable fpts-0" data-column="fpts" data-row-index="0" type="text" placeholder="" value="23.45"></div></div></div></div></div></span></div></div><div class="fixedDataTableCellLayout_main public_fixedDataTableCell_main" role="gridcell" style="height: 36px; width: 65px; left: 245px;"><div><span data-tip="false" data-for="rg-projection-projOwn-0"><div class="fixedDataTableCellLayout_wrap1 public_fixedDataTableCell_wrap1 input-cell"><div class="fixedDataTableCellLayout_wrap2 public_fixedDataTableCell_wrap2"><div class="fixedDataTableCellLayout_wrap3 public_fixedDataTableCell_wrap3"><div class="public_fixedDataTableCell_cellContent"><div class="input-editable-container"><input class="input-editable projOwn-0" data-column="projOwn" data-row-index="0" type="text" placeholder="" value="12.7">%</div></div></div></div></div></span></div></div><div class="fixedDataTableCellLayout_main public_fixedDataTableCell_main" role="gridcell" style="height: 36px; width: 55px; left: 310px;"><div><span data-tip="false" data-for="rg-projection-floor-0"><div class="fixedDataTableCellLayout_wrap1 public_fixedDataTableCell_wrap1 input-cell"><div class="fixedDataTableCellLayout_wrap2 public_fixedDataTableCell_wrap2"><div class="fixedDataTableCellLayout_wrap3 public_fixedDataTableCell_wrap3"><div class="public_fixedDataTableCell_cellContent"><div class="input-editable-container"><input class="input-editable floor-0" data-column="floor" data-row-index="0" type="text" placeholder="" value="11.02"></div></div></div></div></div></span></div></div></div></div><div class="fixedDataTableRowLayout_fixedColumnsDivider public_fixedDataTableRow_fixedColumnsDivider" style="left: 309px; height: 36px;"></div><div class="fixedDataTableCellGroupLayout_cellGroupWrapper" style="height: 36px; width: 172px; left: 658px;"><div class="fixedDataTableCellGroupLayout_cellGroup" style="height: 36px; position: absolute; width: 172px; z-index: 2; transform: translate3d(0px, 0px, 0px);"><div class="fixedDataTableCellLayout_main public_fixedDataTableCell_main" role="gridcell" style="height: 36px; width: 28px; left: 0px;"><div class="fixedDataTableCellLayout_wrap1 public_fixedDataTableCell_wrap1"><div class="fixedDataTableCellLayout_wrap2 public_fixedDataTableCell_wrap2"><div class="fixedDataTableCellLayout_wrap3 public_fixedDataTableCell_wrap3"><div class="public_fixedDataTableCell_cellContent"><button type="button" class="icn icn-menu2" data-event="focus" data-tip="true" data-for="group-select-tooltip" data-row-index="0" style="padding: 2px; cursor: pointer;"></button></div></div></div></div></div><div class="fixedDataTableCellLayout_main public_fixedDataTableCell_main" role="gridcell" style="height: 36px; width: 32px; left: 28px;"><div class="fixedDataTableCellLayout_wrap1 public_fixedDataTableCell_wrap1"><div class="fixedDataTableCellLayout_wrap2 public_fixedDataTableCell_wrap2"><div class="fixedDataTableCellLayout_wrap3 public_fixedDataTableCell_wrap3"><div class="public_fixedDataTableCell_cellContent"><button type="button" data-row-index="0" class="player-lock-button circle-toggle-icn off"><span class="icn icn icn-unlock"></span></button></div></div></div></div></div><div class="fixedDataTableCellLayout_main public_fixedDataTableCell_main" role="gridcell" style="height: 36px; width: 41px; left: 60px;"><div class="fixedDataTableCellLayout_wrap1 public_fixedDataTableCell_wrap1 input-cell"><div class="fixedDataTableCellLayout_wrap2 public_fixedDataTableCell_wrap2"><div class="fixedDataTableCellLayout_wrap3 public_fixedDataTableCell_wrap3"><div class="public_fixedDataTableCell_cellContent"><div class="input-editable-container"><input class="input-editable minLiked-0" data-column="minLiked" data-row-index="0" type="text" placeholder="0" value=""></div></div></div></div></div></div><div class="fixedDataTableCellLayout_main public_fixedDataTableCell_main" role="gridcell" style="height: 36px; width: 41px; left: 101px;"><div class="fixedDataTableCellLayout_wrap1 public_fixedDataTableCell_wrap1 input-cell"><div class="fixedDataTableCellLayout_wrap2 public_fixedDataTableCell_wrap2"><div class="fixedDataTableCellLayout_wrap3 public_fixedDataTableCell_wrap3"><div class="public_fixedDataTableCell_cellContent"><div class="input-editable-container"><input class="input-editable maxLiked-0" data-column="maxLiked" data-row-index="0" type="text" placeholder="100" value=""></div></div></div></div></div></div><div class="fixedDataTableCellLayout_main public_fixedDataTableCell_main" role="gridcell" style="height: 36px; width: 30px; left: 142px;"><div class="fixedDataTableCellLayout_wrap1 public_fixedDataTableCell_wrap1"><div class="fixedDataTableCellLayout_wrap2 public_fixedDataTableCell_wrap2"><div class="fixedDataTableCellLayout_wrap3 public_fixedDataTableCell_wrap3"><div class="public_fixedDataTableCell_cellContent"><button class="player-exclude-button circle-toggle-icn off" data-row-index="0" type="button"><span data-row-index="0" class="icn icn-delete"></span></button></div></div></div></div></div></div></div><div class="fixedDataTableRowLayout_columnsShadow fixedDataTableRowLayout_columnsRightShadow fixedDataTableRowLayout_fixedColumnsDivider public_fixedDataTableRow_columnsShadow public_fixedDataTableRow_columnsRightShadow public_fixedDataTableRow_fixedColumnsDivider" style="height: 36px; left: 653px;"></div><div class="public_fixedDataTable_scrollbarSpacer" style="width: 15px; height: 36px; left: 828px;"></div></div><div class="fixedDataTableRowLayout_columnsShadow fixedDataTableRowLayout_columnsRightShadow public_fixedDataTableRow_columnsShadow public_fixedDataTableRow_columnsRightShadow" style="height: 36px;"></div></div></div>

In [ ]:
<div class="fixedDataTableCellGroupLayout_cellGroup" style="height: 36px; position: absolute; width: 3121px; z-index: 0; transform: translate3d(0px, 0px, 0px);"><div class="fixedDataTableCellLayout_main public_fixedDataTableCell_main" role="gridcell" style="height: 36px; width: 74px; left: 0px;"><div class="fixedDataTableCellLayout_wrap1 public_fixedDataTableCell_wrap1 col-group-first"><div class="fixedDataTableCellLayout_wrap2 public_fixedDataTableCell_wrap2"><div class="fixedDataTableCellLayout_wrap3 public_fixedDataTableCell_wrap3"><div class="public_fixedDataTableCell_cellContent"><span>RB</span></div></div></div></div></div><div class="fixedDataTableCellLayout_main public_fixedDataTableCell_main" role="gridcell" style="height: 36px; width: 51px; left: 74px;"><div class="fixedDataTableCellLayout_wrap1 public_fixedDataTableCell_wrap1" style="white-space: nowrap;"><div class="fixedDataTableCellLayout_wrap2 public_fixedDataTableCell_wrap2"><div class="fixedDataTableCellLayout_wrap3 public_fixedDataTableCell_wrap3"><div class="public_fixedDataTableCell_cellContent">NYG</div></div></div></div></div><div class="fixedDataTableCellLayout_main public_fixedDataTableCell_main" role="gridcell" style="height: 36px; width: 65px; left: 125px;"><div class="fixedDataTableCellLayout_wrap1 public_fixedDataTableCell_wrap1 col-group-last" style="white-space: nowrap;"><div class="fixedDataTableCellLayout_wrap2 public_fixedDataTableCell_wrap2"><div class="fixedDataTableCellLayout_wrap3 public_fixedDataTableCell_wrap3"><div class="public_fixedDataTableCell_cellContent">@ CHI</div></div></div></div></div><div class="fixedDataTableCellLayout_main public_fixedDataTableCell_main" role="gridcell" style="height: 36px; width: 55px; left: 190px;"><div><span data-tip="false" data-for="rg-projection-fpts-1"><div class="fixedDataTableCellLayout_wrap1 public_fixedDataTableCell_wrap1 input-cell col-group-first"><div class="fixedDataTableCellLayout_wrap2 public_fixedDataTableCell_wrap2"><div class="fixedDataTableCellLayout_wrap3 public_fixedDataTableCell_wrap3"><div class="public_fixedDataTableCell_cellContent"><div class="input-editable-container"><input class="input-editable fpts-1" data-column="fpts" data-row-index="1" type="text" placeholder="" value="19.82"></div></div></div></div></div></span></div></div><div class="fixedDataTableCellLayout_main public_fixedDataTableCell_main" role="gridcell" style="height: 36px; width: 65px; left: 245px;"><div><span data-tip="false" data-for="rg-projection-projOwn-1"><div class="fixedDataTableCellLayout_wrap1 public_fixedDataTableCell_wrap1 input-cell"><div class="fixedDataTableCellLayout_wrap2 public_fixedDataTableCell_wrap2"><div class="fixedDataTableCellLayout_wrap3 public_fixedDataTableCell_wrap3"><div class="public_fixedDataTableCell_cellContent"><div class="input-editable-container"><input class="input-editable projOwn-1" data-column="projOwn" data-row-index="1" type="text" placeholder="" value="3">%</div></div></div></div></div></span></div></div><div class="fixedDataTableCellLayout_main public_fixedDataTableCell_main" role="gridcell" style="height: 36px; width: 55px; left: 310px;"><div><span data-tip="false" data-for="rg-projection-floor-1"><div class="fixedDataTableCellLayout_wrap1 public_fixedDataTableCell_wrap1 input-cell"><div class="fixedDataTableCellLayout_wrap2 public_fixedDataTableCell_wrap2"><div class="fixedDataTableCellLayout_wrap3 public_fixedDataTableCell_wrap3"><div class="public_fixedDataTableCell_cellContent"><div class="input-editable-container"><input class="input-editable floor-1" data-column="floor" data-row-index="1" type="text" placeholder="" value="8.13"></div></div></div></div></div></span></div></div></div>

In [272]:
pprint(dir(driver))

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_file_detector',
 '_is_remote',
 '_mobile',
 '_switch_to',
 '_unwrap_value',
 '_web_element_cls',
 '_wrap_value',
 'add_cookie',
 'application_cache',
 'back',
 'capabilities',
 'close',
 'command_executor',
 'create_options',
 'create_web_element',
 'current_url',
 'current_window_handle',
 'delete_all_cookies',
 'delete_cookie',
 'desired_capabilities',
 'error_handler',
 'execute',
 'execute_async_script',
 'execute_script',
 'file_detector',
 'file_detector_context',
 'find_element',
 'find_element_by_class_name',
 'find_element_by_css_selector',
 'find_element_by_id',
 'find_element_by_link_text',
 'find_element_by_

In [275]:
soup

<html class="user"><head>
<title>DraftKings NFL Lineup Optimizer</title>
<meta content="Daily Fantasy Lineup Optimizer for DraftKings NFL" name="description"/>
<meta content="initial-scale=1.0, width=device-width, maximum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<link href="/favicon.ico" rel="icon" type="image/x-icon"/>
<link href="https://rotogrinders.com/lineuphq/nfl?site=draftkings" rel="canonical"/>
<!-- Google Tag Manager -->
<noscript>
<iframe height="0" src="//www.googletagmanager.com/ns.html?id=GTM-PZPRPW" style="display:none;visibility:hidden" width="0"></iframe>
</noscript>
<script async="" id="www-widgetapi-script" src="https://s.ytimg.com/yts/jsbin/www-widgetapi-vflZtsE8X/www-widgetapi.js" type="text/javascript"></script><script async="" id="undefined" src="https://www.google-analytics.com/plugins/ga/inpage_linkid.js" type="text/javascript"></script><script async="" src="https://www.google-analytics.com/analyti

In [277]:
a = soup.findAll('input-editable-container')
for b in a:
    print(b)

In [294]:
pprint(dir(soup.findAll("div", {"class": "input-editable-container"})[0]))

['__bool__',
 '__call__',
 '__class__',
 '__contains__',
 '__copy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_all_strings',
 '_find_all',
 '_find_one',
 '_is_xml',
 '_lastRecursiveChild',
 '_last_descendant',
 '_should_pretty_print',
 'append',
 'attrs',
 'can_be_empty_element',
 'cdata_list_attributes',
 'childGenerator',
 'children',
 'clear',
 'contents',
 'decode',
 'decode_contents',
 'decompose',
 'descendants',
 'encode',
 'encode_contents',
 'extend',
 'extract',
 'fetchNextSiblings',
 'fetchParents',
 'fetchPrevious',
 'fetchPreviousSiblings',
 'find',
 'findAl

In [299]:
soup.findAll("div", {"class": "input-editable-container"})[0].decode()

'<div class="input-editable-container"><input class="input-editable fpts-0" data-column="fpts" data-row-index="0" placeholder="" type="text" value="26.16"/></div>'

In [301]:
import pickle
filehandler = open("../resources/latest_results.pkl","wb")
pickle.dump(latest,filehandler)
filehandler.close()